<a href="https://colab.research.google.com/github/Taazkir/ML-predicting-house-prices-philadelphia/blob/main/Predicting_House_Prices_in_Philadelphia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
# https://medium.com/@GaussEuler/philadelphia-housing-data-part-i-data-analysis-fe45415554a9

# Mount Google Drive
drive.mount('/content/drive')

import pandas as pd
# Read the CSV file
df = pd.read_csv('opa_properties_public.csv')
df



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-a75c2e8f3464>:9: DtypeWarning: Columns (5,26,49,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('opa_properties_public.csv')


,the_geom,the_geom_webmercator,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,...,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid,lat,lng
0,0101000020E610000029C6C53D2BCC52C0DD31054E9CF7...,0101000020110F0000D6E1D100F4ED5FC11AC63D999986...,2022-05-24,NaN,SWC OF DICKINSON ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,...,NaN,NaN,19146.0,RSA5,1001202566,NaN,NaN,437224013,39.934458,-75.190139
1,0101000020E6100000B08BC01062CB52C0B80FFF2AA3FF...,0101000020110F0000F67827499EEC5FC1F75C95A67E8F...,2022-05-24,NaN,NEC DOVER ST,NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,...,NaN,NaN,19132.0,CMX1,1001321879,NaN,NaN,437224014,39.997167,-75.177860
2,0101000020E61000001A1448DC39CB52C0DD45FFD4CBF4...,0101000020110F000094715BFE59EC5FC164CB68B77B83...,2022-05-24,NaN,NWC 16TH ST,NaN,T38,ROW B/OFF-STR 2 STY STONE,3,MIXED USE,...,1950.0,NaN,19145.0,CMX1,1001408261,820,ROW MIXED-COM/RES-BLT AS RES,437224015,39.912470,-75.175407
3,0101000020E610000084BBFD1FCCCB52C0785516F4F9FE...,0101000020110F0000150E447052ED5FC12914760FC38E...,2022-05-24,NaN,"343'7 1/8"" W 31ST ST",NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,...,NaN,NaN,19132.0,RSA5,1001168670,NaN,NaN,437224016,39.992003,-75.184334
4,0101000020E6100000D94623BCE3CA52C06001DBCC5EFB...,0101000020110F00009EC95DB3C7EB5FC1430D34E8C38A...,2022-05-24,NaN,197' W OF 19TH ST,NaN,ZL0,MISC FUNERAL HOME MASONRY,4,COMMERCIAL,...,1920.0,NaN,19130.0,RM1,1001494378,246,FUNERAL HOME,437224017,39.963831,-75.170150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582928,0101000020E6100000F99474516DC852C00154F0E284FD...,0101000020110F0000DB9FD8DF98E75FC1B8B0C387258D...,2022-05-24,NaN,"258'3 1/8"" N BERGES",NaN,O50,ROW 3 STY MASONRY,13,VACANT LAND - RESIDENTIAL,...,NaN,NaN,19125.0,RSA5,1001677168,NaN,NaN,437804312,39.980618,-75.131672
582929,0101000020E61000003605B9853DCA52C035604A8C7402...,0101000020110F0000A65F345FADEA5FC1530836879E92...,2022-05-24,NaN,NEC CLASISSA ST,0.0,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,...,NaN,NaN,19140.0,RSA5,1001175827,NaN,NaN,437804313,40.019182,-75.160005
582930,0101000020E6100000A436B2114ACB52C01785F607EAF8...,0101000020110F0000C6308E8675EC5FC11B93A8440B88...,2023-04-18,NaN,329' W 20TH ST,0.0,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,...,1925.0,NaN,19146.0,RSA5,1001299142,22,ROW TYPICAL,437804314,39.944642,-75.176396
582931,0101000020E6100000E382BDAFD0CA52C0690135674AFD...,0101000020110F0000A2805058A7EB5FC1D1E8C9B5E48C...,2022-05-24,C,"332'10"" W 20TH ST",0.0,O50,ROW 3 STY MASONRY,1,SINGLE FAMILY,...,1915.0,NaN,19121.0,RSA5,1001407218,22,ROW TYPICAL,437804315,39.978833,-75.168987
